In [11]:
import glob
import json
import pandas as pd

SKLEARN_DATA = "../data/sklearn/modules/sklearn_modules.json"

ALL_SKLEARN_PROJECTS = "statistics/sklearn/statistics/*"
ALL_TENSORFLOW_PROJECTS = "statistics/tensorflow/statistics/*"
ALL_PYTORCH_PROJECTS = "statistics/pytorch/statistics/*"

In [12]:
projects = glob.glob(ALL_SKLEARN_PROJECTS)
print("Len projects: ", len(projects))

Len projects:  2


In [13]:
with open(SKLEARN_DATA, "r", encoding="utf-8") as sklearn_file:
    sklearn_data = json.load(sklearn_file)

data = []

for project in glob.glob(ALL_SKLEARN_PROJECTS):
    project_name = project.split("\\")[-1].split(".")[0]
    project_name = project_name.replace("statistics_", "")

    with open(project, "r", encoding="utf-8") as project_file:
        project_data = json.load(project_file)

    for file in project_data:
        file_data = project_data[file]

        for module in file_data:
            module_name = module.split("_")[0]
            module_data = file_data[module]

            sklearn_module = next(filter(lambda x: x["name"] == module_name, sklearn_data))

            total_options = len(sklearn_module["params"])

            if module_name == "KMeans":
                data.append({
                    "project": project_name,
                    "total_options": total_options,
                    "options_used": len(module_data),
                    "portion": len(module_data)/total_options
                })

PermissionError: [Errno 13] Permission denied: 'statistics/sklearn\\results'

In [ ]:
modules = set()

for project in glob.glob(ALL_SKLEARN_PROJECTS):
    project_name = project.split("\\")[-1].split(".")[0]
    project_name = project_name.replace("statistics_", "")

    with open(project, "r", encoding="utf-8") as project_file:
        project_data = json.load(project_file)

    for file in project_data:
        file_data = project_data[file]

        for module in file_data:
            module_name = module.split("_")[0]
            modules.add(module_name)


print("Modules: ", modules)
print("Len modules: ", len(modules))


Modules:  {'TruncatedSVD', 'KDTree', 'OneClassSVM', 'ShuffleSplit', 'IsotonicRegression', 'OneHotEncoder', 'GaussianMixture', 'SimpleImputer', 'SVR', 'RandomForestClassifier', 'MultinomialNB', 'PCA', 'LinearRegression', 'ExtraTreesClassifier', 'LabelEncoder', 'TfidfVectorizer', 'TransformedTargetRegressor', 'CountVectorizer', 'MLPRegressor', 'AgglomerativeClustering', 'BayesianGaussianMixture', 'ParameterSampler', 'RFE', 'SVC', 'SGDOneClassSVM', 'LabelBinarizer', 'GridSearchCV', 'DBSCAN', 'HuberRegressor', 'KFold', 'RBFSampler', 'TfidfTransformer', 'GroupKFold', 'MultiLabelBinarizer', 'ExtraTreesRegressor', 'OneVsRestClassifier', 'Perceptron', 'BernoulliNB', 'KBinsDiscretizer', 'DecisionTreeRegressor', 'ComplementNB', 'LatentDirichletAllocation', 'KNeighborsClassifier', 'RepeatedStratifiedKFold', 'KMeans', 'GroupShuffleSplit', 'TSNE', 'RANSACRegressor', 'RobustScaler', 'StratifiedKFold', 'MultiOutputRegressor', 'GradientBoostingRegressor', 'DummyRegressor', 'SGDRegressor', 'RepeatedKFo

In [ ]:
all_data = {}

for name in modules:
    data = []

    for project in glob.glob(ALL_SKLEARN_PROJECTS):
        project_name = project.split("\\")[-1].split(".")[0]
        project_name = project_name.replace("statistics_", "")

        with open(project, "r", encoding="utf-8") as project_file:
            project_data = json.load(project_file)

        for file in project_data:
            file_data = project_data[file]

            for module in file_data:
                default_counter = 0
                custom_counter = 0
                module_name = module.split("_")[0]
                module_data = file_data[module]

                sklearn_module = next(filter(lambda x: x["name"] == module_name, sklearn_data))

                total_options = len(sklearn_module["params"])

                if module_name == name:
                    if module_data:
                        for option in module_data:
                            try:
                                param = next(filter(lambda x: x[0] == option, sklearn_module["params"]))
                                default_value = param[1].split("=")[-1]
                                option_value = module_data[option]   
                                if default_value == option_value:
                                    default_counter += 1
                                else:
                                    custom_counter +=1
                            except:
                                custom_counter +=1

                    data.append({
                        "project": project_name,
                        "file": file,
                        "total_options": total_options,
                        "options_used": len(module_data),
                        "default_options_used": default_counter,
                        "custom_options_used": custom_counter
                    })

    all_data[name] = data


with open("sklearn_modules_data.json", "w") as outfile:
    json.dump(all_data, outfile, sort_keys=True, indent=4)